In [4]:
# !pip install python-binance

from market_imbalance import MarketImbalance
from market_trades import MarketTrade
from find_imbalances import sort_imbalances_by_timestamp
from plot_market_candles_with_plotly import plot_market_candles_with_plotly, plot_market_candles_with_imbalances
from market_statistics import conversion_factors, calculate_statistics, plot_fill_time_histogram
from convert_ts_to_datetime import format_elapsed_time
from interact_with_binance import adjust_timestamps_to_local

import os
from interact_with_binance import fetch_ohlcv, fetch_ohlcv_as_df
from market_candles import MarketCandle
from binance.client import Client
from convert_ts_to_datetime import convert_ts_to_datetime
from datetime import date, timedelta, datetime
import datetime

def find_imbalances_after_fall(historical_candles):
    imbalances_after_fall = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.low_price > next_candle.high_price:
            delta_to_be_filled_in = prev_candle.low_price - next_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_fall",
                timestamp=current_candle.timestamp,
                open_price=next_candle.high_price,
                close_price=prev_candle.low_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_fall.append(imbalance)
    return imbalances_after_fall

def find_imbalances_after_rise(historical_candles):
    imbalances_after_rise = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.high_price < next_candle.low_price:
            delta_to_be_filled_in = next_candle.low_price - prev_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_rise",
                timestamp=current_candle.timestamp,
                open_price=next_candle.low_price,
                close_price=prev_candle.high_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_rise.append(imbalance)
    return imbalances_after_rise

def check_if_imbalance_filled(imbalances, candles):
    for imbalance in imbalances:
        if imbalance.is_full_filled:
            continue  # Passer les imbalances déjà comblés

        # Ne vérifier que les bougies avec un timestamp >= à celui de l'imbalance
        relevant_candles = [candle for candle in candles if candle.timestamp > imbalance.timestamp]

        for candle in relevant_candles:
            if imbalance.imbalance_type == "imbalance_after_fall":
                # Vérifier si la bougie comble l'imbalance après une chute
                if candle.high_price >= imbalance.close_price:
                    imbalance.is_full_filled = True
                    imbalance.was_fullfilled_at = candle.timestamp
                    imbalance.candles_of_fullfilling = candle

                    # Calculer le temps pour combler l'imbalance
                    time_to_fullfill = candle.timestamp - imbalance.timestamp
                    imbalance.time_to_be_fullfilled = time_to_fullfill  # en millisecondes
                    break  # Sortir de la boucle une fois comblé

            elif imbalance.imbalance_type == "imbalance_after_rise":
                # Vérifier si la bougie comble l'imbalance après une hausse
                if candle.low_price <= imbalance.close_price:
                    imbalance.is_full_filled = True
                    imbalance.was_fullfilled_at = candle.timestamp
                    imbalance.candles_of_fullfilling = candle

                    # Calculer le temps pour combler l'imbalance
                    time_to_fullfill = candle.timestamp - imbalance.timestamp
                    imbalance.time_to_be_fullfilled = time_to_fullfill  # en millisecondes
                    break  # Sortir de la boucle une fois comblé

def get_unfilled_imbalances(imbalances):
    unfilled_imbalances = [imbalance for imbalance in imbalances if not imbalance.is_full_filled]
    return unfilled_imbalances

def get_fullfilled_imbalances(imbalances):
    fullfilled_imbalances = [imbalance for imbalance in imbalances if imbalance.is_full_filled]
    return fullfilled_imbalances

def calculate_fulfillment_percentage(total_imbalances, fullfilled_imbalances):
    if total_imbalances == 0:
        raise ValueError("Le nombre total d'imbalances ne peut pas être zéro.")

    # Calcule le pourcentage d'imbalances comblés
    percentage = (len(fullfilled_imbalances) / len(total_imbalances)) * 100
    
    return round(percentage, 2)  # Arrondir à deux décimales

# Define time units: millisecond, second, minute, hour, and day
millisecond = 1
second = millisecond * 1000
minute = 60 * second
hour = 60 * minute
day = 24 * hour

In [7]:
### This section is about fetching historical data from the Binance REST API.###

api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

# Récupérer l'historique du marché sur Binance
interval_value = Client.KLINE_INTERVAL_1MINUTE
from_date = "12 mai 2024"
symbol = "BTCUSDT"

historical_candles = fetch_ohlcv(client, symbol = symbol, interval=interval_value, from_date=from_date) #"1 Jan, 2015"
# historical_candles_df = fetch_ohlcv_as_df(client, symbol = symbol, interval=interval_value, from_date=from_date) #"1 Jan, 2015"
# adjust_timestamps_to_local(historical_candles_df, +2)

print(len(historical_candles), "bougies récupérées depuis le", from_date)
previous_candle_timestamp = int(historical_candles[-1].timestamp)
previous_candle_datetime = datetime.datetime.fromtimestamp(previous_candle_timestamp/1000)
print('date de la dernière bougie récupérée :', previous_candle_datetime)


908 bougies récupérées depuis le 12 mai 2024
date de la dernière bougie récupérée : 2024-05-12 17:07:00


In [8]:
'''### This section is about fetching real-time data from the Binance Websocket API.###

import nest_asyncio
import asyncio
import datetime
from binance import AsyncClient, BinanceSocketManager

# Apply nest_asyncio to avoid event loop errors
nest_asyncio.apply()

# Main function to retrieve the latest trades on Binance
async def main():
    client = await AsyncClient.create()  # Create an asynchronous client
    bm = BinanceSocketManager(client)  # Create a socket manager
    ts = bm.trade_socket('BTCUSDT')  # Create a trade socket for the BTC/USDT symbol
    
    responses = []  # List to store responses

    # Use the socket to receive messages
    async with ts as tscm: 
        while len(responses) < 1600:  # Collect trades
            res = await tscm.recv()  # Get a message from the socket
            responses.append(res)  # Add the message to the list
            
    await client.close_connection()  # Close connection with Binance
    return responses  # Return collected messages

# Script execution
if __name__ == "__main__":
    loop = asyncio.get_event_loop()  # Get the event loop
    trades = loop.run_until_complete(main())  # Execute the main function
    trades = [MarketTrade(trade) for trade in trades]  # Assuming MarketTrade class exists'''


'### This section is about fetching real-time data from the Binance Websocket API.###\n\nimport nest_asyncio\nimport asyncio\nimport datetime\nfrom binance import AsyncClient, BinanceSocketManager\n\n# Apply nest_asyncio to avoid event loop errors\nnest_asyncio.apply()\n\n# Main function to retrieve the latest trades on Binance\nasync def main():\n    client = await AsyncClient.create()  # Create an asynchronous client\n    bm = BinanceSocketManager(client)  # Create a socket manager\n    ts = bm.trade_socket(\'BTCUSDT\')  # Create a trade socket for the BTC/USDT symbol\n    \n    responses = []  # List to store responses\n\n    # Use the socket to receive messages\n    async with ts as tscm: \n        while len(responses) < 1600:  # Collect trades\n            res = await tscm.recv()  # Get a message from the socket\n            responses.append(res)  # Add the message to the list\n            \n    await client.close_connection()  # Close connection with Binance\n    return responses

In [10]:
import nest_asyncio
import asyncio
import datetime
from binance import AsyncClient, BinanceSocketManager

# Apply nest_asyncio to avoid event loop errors
nest_asyncio.apply()

# Main function to retrieve the latest trades on Binance
async def main(previous_candle_timestamp = previous_candle_timestamp):
    client = await AsyncClient.create()  # Create an asynchronous client
    bm = BinanceSocketManager(client)  # Create a socket manager
    ts = bm.trade_socket('BTCUSDT')  # Create a trade socket for the BTC/USDT symbol
    
    # Use the socket to receive messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()  # Get a message from the socket
            trade = MarketTrade(res)
                        
            ### This section is about updating the forming candle ###
            # Je crée la bougie avec des données de base
            forming_candle = MarketCandle(
                    timestamp = previous_candle_timestamp + 1*minute, # int
                    open_price = 0, # float
                    high_price = 0, # float
                    low_price = 0, # float
                    close_price = 0, # float
                    volume = 0, # float
                    close_time = previous_candle_timestamp + 2*minute - 1*millisecond, # int
                    quote_asset_volume = 0, # float
                    number_of_trades = 0, # int
                    taker_buy_base_asset_volume = 0, # float
                    taker_buy_quote_asset_volume = 0, # float
                    ignore = None)
            
            # J'analyse les trades pour mettre à jour les valeurs de la bougie
            trades_in_the_interval = []
            
            if forming_candle.open_price == 0 and trade.trade_time > forming_candle.timestamp :
                forming_candle.open_price = trade.price
            
            if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time:
                trades_in_the_interval.append(trade.price)

            if trade.trade_time >= forming_candle.close_time :
                forming_candle.close_price = trades_in_the_interval[-1]
                
            if trade.trade_time >= forming_candle.timestamp and trade.trade_time <= forming_candle.close_time and trade.price >= forming_candle.high_price:
                forming_candle.high_price = trade.price

            if trade.trade_time >= forming_candle.timestamp and trade.trade_time <= forming_candle.close_time and trade.price <= forming_candle.open_price:
                forming_candle.low_price = trade.price
            
            if not trade.is_the_buyer_the_market_maker :
                forming_candle.taker_buy_base_asset_volume += trade.quantity
                forming_candle.taker_buy_quote_asset_volume += trade.quantity * trade.price
                
            forming_candle.volume += trade.quantity
            forming_candle.quote_asset_volume += trade.quantity * trade.price
            forming_candle.number_of_trades = len(trades_in_the_interval)

            print('candle.timestamp :', datetime.datetime.fromtimestamp(forming_candle.timestamp/1000))
            print('candle.open_price :', forming_candle.open_price)
            print('candle.high_price :', forming_candle.high_price)
            print('candle.low_price :', forming_candle.low_price)
            print('candle.close_price :', forming_candle.close_price)
            print('candle.number_of_trades :', forming_candle.number_of_trades)
            print()

            if forming_candle.close_price != 0 :
                historical_candles.append(forming_candle)
                previous_candle_timestamp += 1*minute
        
    await client.close_connection()  # Close connection with Binance
    return responses  # Return collected messages

# Script execution
if __name__ == "__main__":
    loop = asyncio.get_event_loop()  # Get the event loop
    trades = loop.run_until_complete(main())  # Execute the main function
    trades = [MarketTrade(trade) for trade in trades]  # Assuming MarketTrade class exists


candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.01
candle.high_price : 61194.01
candle.low_price : 61194.01
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.01
candle.high_price : 61194.01
candle.low_price : 61194.01
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.01
candle.high_price : 61194.01
candle.low_price : 61194.01
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.01
candle.high_price : 61194.01
candle.low_pri

KeyboardInterrupt: 

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.0
candle.high_price : 61194.0
candle.low_price : 61194.0
candle.close_price : 0.0
candle.number_of_trades : 1

candle.timestamp : 2024-05-12 17:08:00
candle.open_price : 61194.01
candle.high_price : 61194.01
candle.low_price : 6119

CANCEL read_loop


In [9]:
# Je souhaite créer une nouvelle bougie à partir des trades que je reçois
historical_candles_initial_lenght = len(historical_candles)
print(len(historical_candles))

# while len(historical_candles) <= historical_candles_initial_lenght + 5 :

while forming_candle.timestamp > trades[]
    ### This section is about updating the forming candle ###
    # Je crée la bougie avec des données de base
    forming_candle = MarketCandle(
            timestamp = previous_candle_timestamp + 1*minute, # int
            open_price = 0, # float
            high_price = 0, # float
            low_price = 0, # float
            close_price = 0, # float
            volume = 0, # float
            close_time = previous_candle_timestamp + 2*minute - 1*millisecond, # int
            quote_asset_volume = 0, # float
            number_of_trades = 0, # int
            taker_buy_base_asset_volume = 0, # float
            taker_buy_quote_asset_volume = 0, # float
            ignore = None)

    # J'analyse les trades pour mettre à jour les valeurs de la bougie
    trades_in_the_interval = []
    for trade in trades :
        
        if forming_candle.open_price == 0 and trade.trade_time > forming_candle.timestamp :
            forming_candle.open_price = trade.price
        
        if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time:
            trades_in_the_interval.append(trade.price)

        if trade.trade_time >= forming_candle.close_time :
            forming_candle.close_price = trades_in_the_interval[-1]
            
        if trade.trade_time >= forming_candle.timestamp and trade.trade_time <= forming_candle.close_time and trade.price >= forming_candle.high_price:
            forming_candle.high_price = trade.price

        if trade.trade_time >= forming_candle.timestamp and trade.trade_time <= forming_candle.close_time and trade.price <= forming_candle.open_price:
            forming_candle.low_price = trade.price
        
        if not trade.is_the_buyer_the_market_maker :
            forming_candle.taker_buy_base_asset_volume += trade.quantity
            forming_candle.taker_buy_quote_asset_volume += trade.quantity * trade.price
            
        forming_candle.volume += trade.quantity
        forming_candle.quote_asset_volume += trade.quantity * trade.price
        forming_candle.number_of_trades = len(trades_in_the_interval)

    print('candle.timestamp :', datetime.datetime.fromtimestamp(forming_candle.timestamp/1000))
    print('candle.open_price :', forming_candle.open_price)
    print('candle.high_price :', forming_candle.high_price)
    print('candle.low_price :', forming_candle.low_price)
    print('candle.close_price :', forming_candle.close_price)
    print('candle.number_of_trades :', forming_candle.number_of_trades)
    print()

    historical_candles.append(forming_candle)
    previous_candle_timestamp += 1*minute

len(historical_candles)

885
candle.timestamp : 2024-05-12 16:45:00
candle.open_price : 61167.37
candle.high_price : 61167.38
candle.low_price : 61132.01
candle.close_price : 61132.01
candle.number_of_trades : 495

886


In [7]:
# Récupérer le timestamp de la dernière bougie dans l'historique
# Récupérer l'intervalle entre le début de chaque bougie
# Convertir l'intervalle en seconde (interval_value_in_sec)
# Créer un nouvel objet bougie
# Définir le timestamp de la bougie actuelle comme égal au timestamp de la bougie précédente + l'interval en seconde
# Définir le timestamp de la bougie suivante Comme égal au timestamp de la bougie actuelle + l'interval en seconde
# Récupérer le dernier trade
# Tant que le timestamp du dernier trade est supérieur ou égal au timestamp de la bougie actuelle 
# et inférieur ou égal au timestamp de la bougie suivante : 
# alors vérifier si le prix d'ouverture de la bougie est null :
# si le prix d'ouverture est null, alors définir le prix d'ouverture de la bougie actuelle comme égale au prix du dernier trade

In [8]:
# J'ai besoin de connaitre la date de chaque trade
# Je dois convertir pour chaque trade le timestamp en dateheure


for trade in trades :
    trade_timestamp = str(trade['T'])
    trade_timestamp = trade_timestamp[:-3] #remove milliseconds
    trade_timestamp = int(trade_timestamp)
    
    trade_datetime = datetime.datetime.fromtimestamp(trade_timestamp)
    trade_minute = trade_datetime.second
    
    print(trade_datetime)
    print(trade_timestamp)
    print(trade_minute)
    
    # maintenant je souhaite assembler les trades qui se déroule la même seconde
    
    # à la base, Je stocke chaque bougie dans une liste d'objet
    # j'isole la dernière bougie.
    
    # Je lance ma boucle pour récupérer les trades en temps réel
    # Je stocke chaque trade dans une liste d'objet
    
    # je requête la dernière bougie uniquement afin de la mettre à jour car certains trades pourraient manquer
    # Je remplace la bougie de base par celle que j'ai reçu
    
    # La dernière bougie a un timestamp.
    # J'identifie la date de cette bougie au format datetime
    # J'isole ensuite la minute, l'heure et le jour
    # Une fois que j'ai la minute de départ :
        tant que la minute
    

    
    

IndentationError: unexpected indent (3458601983.py, line 32)

In [ ]:
for trade in trades : 
    print(trade['E'], trade['p'], trade['q'])
    
# faire une analyse du nombre de trade par minute.
# Est ce que le nombre de trade prédisent des moments de forte hausse ou de forte baisse ?

In [ ]:
# candle.timestamp = ok
# candle.open_price = ok
# candle.high_price = ok
# candle.low_price = ok
# candle.close_price = ok
# candle.volume = ok
# candle.close_time = ok
# candle.quote_asset_volume = ok
# candle.number_of_trades = ok
# candle.taker_buy_base_asset_volume = ok
# candle.taker_buy_quote_asset_volume =ok
# candle.ignore

In [ ]:
import time
# Assembler les trades en bougies
# historical_candles
# new_candle = MarketCandle()
# historical_candles.append(new_candle)

unix_timestamp_start = 0 #1 janvier 1970 1:00:00
start = 946681200 # 1 janvier 2000 1:00:00

ms = 1000
sec = ms / 1000
min = 60 * sec
hour = 60 * min
day = 24 * hour
year = 365 * day

second_janv_ = unix_timestamp_start + 50*year
second_janv_

today_timestamp = time.time()
today_timestamp
# today_datetime = datetime.datetime(today_timestamp)
    



In [ ]:
# price = res['p']
            # trade_timestamp = res['T']
            # event_ts = res['E']
            # now = round (time.time() * 1000)
            # print(price, now, trade_timestamp, event_ts)
            
            total_amount_traded += float(res['p']) * float(res['q'])
            print(total_amount_traded)